# Svelty
A `Svelt Stores` implementation in python.
See `Svelt Stores` [documentation](https://svelte.dev/docs#run-time-svelte-store) for more information on the original.

## Setup

In [1]:
import nbdev

In [2]:
#| default_exp store

In [3]:
#|export
from __future__ import annotations
from enum import Enum
from types import SimpleNamespace
from typing import List, Callable, TypeVar,  Generic, Sequence, Union, Optional, Any, Set, Tuple, Dict, Protocol, NewType, Literal, TypedDict, overload
from dataclasses import dataclass
from result import Ok, Err, Result #type: ignore

## Svelte Store contract

1. A store must contain a `.subscribe` method, which must accept as its argument a `subscription function`(aka Subscriber or Callback). This `subscription function` must be immediately and synchronously called with the store's current value upon calling `subscribe`. All of a store's active subscription functions must later be synchronously called whenever the store's value changes.

1. The `.subscribe` method must return an `unsubscribe function`(aka Unsubscriber). Calling an `unsubscribe function` must `stop` its subscription, and its corresponding `subscription function` must not be called again by the store.

1. A store may optionally contain a `.set` method, which must accept as its argument a new value for the store, and which synchronously calls all of the store's active subscription functions. Such a store is called a writable store.


~~For interoperability with RxJS Observables, the .subscribe method is also allowed to return an object with an .unsubscribe method, rather than return the unsubscription function directly. Note however that unless .subscribe synchronously calls the subscription (which is not required by the Observable spec), Svelte will see the value of the store as undefined until it does.~~

[Store Contract Documentation](https://svelte.dev/docs#component-format-script-4-prefix-stores-with-$-to-access-their-values-store-contract)

### Types Definition

In [4]:
#| export

T = TypeVar("T")
covT = TypeVar("covT", covariant=True)
Subscriber = Callable[[T], None] # a callback
Unsubscriber = Callable[[], None] # a callback to be used upon termination of the subscription
 
class StoreProtocol(Protocol, Generic[covT]):
    def subscribe(self, subscriber: Subscriber[T]) -> Unsubscriber: ...

## Readable and Writable Stores

### Definition

In [5]:
 
class Base:  #see SimpleNamespace: https://docs.python.org/3/library/types.html
    def __init__(self, /, **kwargs):
        self.__dict__.update(kwargs)
    def __repr__(self):
        items = (f"{k}={v!r}" for k, v in self.__dict__.items())
        return "{}({})".format(type(self).__name__, ", ".join(items))

Updater = Callable[[T], T]

class Store(Base, StoreProtocol[T]):
    value: T
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.value!r})"
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return lambda: None
    def get(self) -> T: return self.value
    def __eq__(self, other):
        if isinstance(self, Store) and isinstance(other, Store):
           return self.get() == other.get()
        return NotImplemented
    def __hash__(self):
        return hash(self.get())
    
class Readable(Store[T]): pass

class Writable(Store[T]):
    set: Subscriber
    update: Optional[Callable[[Updater],None]] = None

### `writable`: a Writable factory


`writable` creates objects of type Writable.  


**Why not implementing as a class?**

Because we may want to protect some attributes.

In [6]:
def writable(initial_value: T) -> Writable[T]:
    value: T = initial_value
    subscribers: Set[Subscriber] = set()

    def subscribe(callback: Subscriber) -> Unsubscriber:
        subscribers.add(callback)
        callback(value)
    
        def unsubscribe() -> None:
            subscribers.remove(callback) if callback in subscribers else None
        return unsubscribe
    
    def _set(new_value: T) -> None:
        nonlocal value
        if new_value != value:
            value = new_value
            for subscriber in subscribers:
                subscriber(value)
    def update(fn: Callable[[T], T]) -> None:
        _set(fn(value))
    return Writable(set=_set, update=update, subscribe=subscribe, get=lambda: value)

### A `Writable` in action

Let's create a `Writable`.  Remember that it returns an unsubscriber function.

In [7]:
a = writable(1)
u1 = a.subscribe(lambda x: print("1:",x))
a

1: 1


Writable(1)

A `writable` can be set from the outside. When it happens, all its subscribers will react.

In [8]:
a.set(2)

1: 2


After unsubscribing, the subscriber should not be called anymore:

In [9]:
u1()
a.set(3)

If you try to unsubscibe twice, it won't break.  It just does nothing the second time.

In [10]:
u1()

In [11]:
u1(),u1()

(None, None)

### `readable`: a ~~Writable~~ Readable factory


`readable` creates objects of type `Readable` by creating a `Writable` and protecting its `set` and `update` methods.  

In [12]:
def readable(initial_value: T) -> Readable[T]:
    res = writable(initial_value)
    return Readable(subscribe=res.subscribe, value=res.value, get=lambda: res.value)

In [13]:
b = readable("foo")
b

Readable('foo')

In [14]:
b.subscribe(lambda x: print("2:",x))

2: foo


<function __main__.writable.<locals>.subscribe.<locals>.unsubscribe() -> 'None'>

We can subscribe to our `readable`, but nothing happens, we cannot `set` a `Readable` from the outside.

In [15]:
try:
    b.set("bar") # should fail
except Exception as error:
  print(error)


'Readable' object has no attribute 'set'


A `store` that does not change is not useful. A `Readable` is like a `writable` where there is only one "thing" from the outside that can change its value. Lets change `writable` to add this "thing", which we will call a `Notifier`.

In [16]:
Notifier = Callable[[Subscriber], Union[Unsubscriber, None]]

In [17]:
def writable(initial_value: T, start: Notifier=lambda x: None) -> Writable[T]:
    value: T = initial_value
    stop: Optional[Unsubscriber] = None
    subscribers: Set[Subscriber] = set()

    def subscribe(callback: Subscriber) -> Unsubscriber:
        subscribers.add(callback)
        if (len(subscribers) == 1):
            nonlocal stop
            stop = start(callback) or (lambda: None)
        callback(value)
    
        def unsubscribe() -> None:
            subscribers.remove(callback) if callback in subscribers else None
            if (len(subscribers) == 0):
                nonlocal stop
                stop() if stop else None
                stop = None
        return unsubscribe
    
    def _set(new_value: T) -> None:
        nonlocal value
        if new_value == value: 
            return None
        value = new_value
        if not stop: # store is not ready yet
            return None
        for subscriber in subscribers:
                subscriber(value)
        
    def update(fn: Callable[[T], T]) -> None:
        _set(fn(value))
    return Writable(start=start, set=_set, update=update, subscribe=subscribe, get=lambda: value)

The previous functionality still works.

In [18]:
del a

In [19]:
a = writable(1)
u1 = a.subscribe(lambda x: print("1:",x))
u2 = a.subscribe(lambda x: print("2:",x))

1: 1
2: 1


In [20]:
a.set(2)

2: 2
1: 2


In [21]:
u1(), u1()

(None, None)

In [22]:
a.set(3)

2: 3


In [23]:
u2(), u2()

(None, None)

In [24]:
a.set(4)

In [25]:
a.get()

4

In [26]:
u1 = a.subscribe(lambda x: print("1:",x))
u2 = a.subscribe(lambda x: print("2:",x))

1: 4
2: 4


In [27]:
u1(), u2(), a.get()

(None, None, 4)

In [29]:
a, a.value, a.get()

(Writable(1), 1, 4)

In [ ]:
y = lambda x: x
b = y(a)

In [ ]:
a, b

But now, we can start the store with a `Notifier` that asynchronously set the value of the store from the outside.

Let's test by creating an asynchronous notifier.

In [ ]:
from threading import Event, Thread

def every(interval, func, *args):
    stopped = Event()
    def loop():
        while not stopped.wait(interval): # the first call is in `interval` secs
            func(*args)
    Thread(target=loop).start()    
    return stopped.set

In [ ]:
def start(set): # notifier
    count = 0
    def incrementCounter():
        nonlocal count
        count = count +1
        set(count)
    cancel = every(1, incrementCounter)
    return cancel

In [ ]:
def myset(x):
    value = x
    print("myset:", value)

In [ ]:
stop = start(myset)

In [ ]:
import time

time.sleep(3)
stop()

In [ ]:
b = writable(0, start)

In [ ]:
b

In [ ]:
u1 = b.subscribe(lambda x: print("1:",x))

In [ ]:
time.sleep(4)
u1()

Nice, it works. Now, let's fix the `Readable` store.

In [ ]:
class Readable(Store[T]):
    value: T
    start: Notifier
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.value!r})"
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return lambda: None
def readable(initial_value: T, start: Notifier) -> Readable[T]:
    res = writable(initial_value, start)
    return Readable(start=start, subscribe=res.subscribe, get=lambda: res.value)

Now, we need to provide a `Notifier` to create a `Readable` store:

In [ ]:
try:
    c = readable(0) # shoud fail
except Exception as error:
    print(error)

### A `Readable` in action

In [ ]:
c = readable(0, start)
c

Notice that while there is no subscribers, the `Notifier` is not started.

In [ ]:
time.sleep(3)
c

The first subscriber starts the `Notifier`.

In [ ]:
stop = c.subscribe(lambda x: print("1:",x))

In [ ]:
time.sleep(3)
stop()

## Derived Store

A `Derived Store` takes a store and transforms it in another store.

In [ ]:
del a, b
a = writable(42)
a

In [ ]:
def derived(source:Store, fn: Updater = lambda x:x) -> Store:
    print('l0:',source)
    def start(set):
        return source.subscribe(lambda x: set(fn(x)))
    return writable(source.value, start)

In [ ]:
b = derived(a)
b, a==b

In [ ]:
u1 = a.subscribe(lambda x: print("a1:",x))
# u2 = b.subscribe(lambda x: print("b1:",x))

In [ ]:
a,b

In [ ]:
a.set(43)

In [ ]:
a,b

In [ ]:
b.set(44)

In [ ]:
def derived(source, fn: Updater) -> Readable:
    print('a:', source)
    def start(set): # notifier
        nonlocal source
        print('b:',source)
        unsubscribe = source.subscribe(lambda x: set(fn(x)))
        return unsubscribe
    print('c:', source)
    return readable(source.value, start)

In [ ]:
del a
a = writable(0)

In [ ]:
u1 = a.subscribe(lambda x: print("a1:",x))

In [ ]:
a.set(1)

In [ ]:
a.value

In [ ]:
b = derived(a, lambda x: x+1)

In [ ]:
a, b

In [ ]:
u2 = b.subscribe(lambda x: print("b1:",x))
# u3 = b.subscribe(lambda x: print("b2:",x))

In [ ]:
a.set(1)

In [ ]:
b

In [ ]:
# U = TypeVar("U")
# Map = Callable[[T], Union[T, U]]
# Derived = Callable[[Store, Map], Store] # derived does not need to emmit the same type as its source

In [ ]:
def start(set): # notifier
    count = 0
    def incrementCounter():
        nonlocal count
        count = count +1
        set(count)
    cancel = every(1, incrementCounter)
    return cancel

In [ ]:
a = writable(0, start)

In [ ]:
a

In [ ]:
stop = a.subscribe(lambda x: print("1:",x))

In [ ]:
stop()

In [ ]:
fn = lambda x: x + 1

In [ ]:
a

In [ ]:
derived()